In [2]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from ramannoodles import peakidentify
from ramannoodles import shoyu
from ramannoodles import spectrafit

In [10]:
def test_peak_assignment():
    """This function tests the operation of the peak_assignment function in peakidentify.py"""
    #First, generate a testing dataset.
    # open spectra library
    shoyu_data_dict = pickle.load(open('../raman_spectra/shoyu_data_dict.p', 'rb'))
    compound_1 = shoyu_data_dict['WATER']
    compound_2 = shoyu_data_dict['CARBON MONOXIDE']
    compound_3 = shoyu_data_dict['CARBON DIOXIDE']
    unknown_x, unknown_y = shoyu.combine_spectra(compound_1, compound_2, plot = False)
    unknown_x = np.asarray(unknown_x)
    unknown_y = np.asarray(unknown_y)
    known_compound_list = [compound_1, compound_2, compound_3]
    precision = 0.03
    
    #Various try statements to make sure that bad inputs are handled correctly. 
    try:
        peak_assignment(1, unknown_y, known_compound_list, precision, False)
    except TypeError:
        print("An invalid unknown_x was passed to the function, and it was handled well with a TypeError.")
    
    
    

In [4]:
def test_compare_unknown_to_known():
    """This function tests the operation of the compare_unknown_to_known function in peakidentify.py"""
    
    
    
    

In [5]:
def test_peak_position_comparisons():
    """This function tests the operation of the peak_position_comparisons function in peakidentify.py"""
    
    

In [6]:
def test_percentage_of_peaks_found():
    """This function tests the operation of the percentage_of_peaks_found function in peakidentify.py"""
    
    

In [7]:
def test_plotting_peak_assignments():
    """This function tests the operation of the peak_assignment function in peakidentify.py"""
    
    